# Logistic regression with constrained parameters

In this notebook, we show how to use the package `PDMP.jl` with a logistic regression problem where the weights are constrained to the positive domain.
This example supports the paper *Piecewise Deterministic Markov Processes for Scalable Monte Carlo on Restricted Domains* by Bierkens et al.
Later (see section B) in the notebook, we show how the results from the paper can be reproduced.

## Part1: demo on how to use the PDMP package

The description below corresponds to the description of the model in the paper.

The data $y_i\in \{-1,1\}$ is modelled by

$$ p(y_i | \xi_i, \beta) = f(y_i\beta^T \xi_i)$$

where $\xi\in\mathbb R^{n\times p}$ are fixed covariates and 

$$f(z)=(1+\exp(-z))^{-1}\in[0,1].$$ 

The parameter $\beta$ is constrained to be in the positive orthant ($\beta_i\ge 0$).

In the experiments, the dimensions are set to $n=10000$ and $p=20$. The entries of $\xi$ and the generating $\beta$ are drawn iid from a uniform on $[0,1]$.

In [1]:
using PDMP

### Declaring the model

#### Basics

In the next cell, we seed the random number generator for reproducibility and define the elements of the model.

Note that:

* the package PDMP implements the logistic function which is used to generate labels.
* we define a proxy for the "$NL$ upper bound", this corresponds to the paper, equation A4 (use of thinning method with a linear upper bound for the logistic regression):

$$ \left\langle \nabla U(\hat x) + N(\nabla U^i(X_t) - \nabla U^i(\hat x)), v\right\rangle^+  \le \langle\nabla U(\hat x), v\rangle^+ + NL\|x-\hat x\| + NL t$$

where $z^+=\max\{0,z\}$. The paper shows that for the logistic regression, the equation holds with

$$ L := {1\over 4}\max_i \|\xi_i\| $$ 

* the data model defined last is an object that encapsulates how the log-likelihood and derived quantities such as the gradient are computed.

In [2]:
srand(1777)

n, p = 10000, 20

ξ = rand(n, p)  # feature matrix
β = rand(p)     # ground truth 

# Generate labels on {-1,+1}
y = (PDMP.logistic.(ξ * β) .> rand(n))
y = y * 2 - 1

# proxy for the N*L upper bound
b = sum( mapslices(ξi->norm(ξi)^2, ξ, 1) )/4

# Declare the Data Model
dm = PDMP.LogReg(ξ, y, b)
;

#### Geometry

We can now define the geometry associated with the model. In particular we want to be able to compute when a particular ray will hit a boundary and which boundary. A ray is understood to be the linear trajectory starting at $x$ and following $x+tv$ for $t>0$.

Here, the domain is simply defined by the non-negativity constraints. Since it is a polygonal domain, we just need to specify the normals to the faces (`ns`) and their intercepts (`a`). In the case of the positive orthant the normals are simply the canonical basis and intercepts are `0`. 

Once the geometry is defined, the `nextboundary` method allows to define the function which, for a given ray, indicates what is the next boundary that will be hit and when.

In [4]:
# Declare the geometry
ns, a = eye(p), zeros(p)
geom  = PDMP.Polygonal(ns, a)

# The geometry defines the 'nextboundary' along a ray
nextboundary(x, v) = PDMP.nextboundary(geom, x, v)
;

#### Events along a ray

Now we need to specify how to handle the Inhomogenous Poisson Process (IPP) associated with the problem. In this case, as detailed in the paper, we consider thinning with a linear upper bound. 

The linear upper bound can be defined via the package using `PDMP.LinearBound`. The method `nextevent_bps` then corresponds to the sampling of the IPP along a ray using the given method.

**Note**: other kernels can be used here (e.g.: zig-zag), please refer to the package documentation for more information. 

In [6]:
# Gradients and sampling of IPP
gll_cv       = PDMP.gradloglik_cv(dm, β)
gll_star     = PDMP.gradloglik(dm, β)
linear_bound = PDMP.LinearBound(β, gll_star, dm.b)

# define how to compute the next event along a ray
nextevent(x, v) = PDMP.nextevent_bps(linear_bound, x, v)
;

### Defining the simulation

We now need to define the simulation parameters:

Stopping criterions:
* if event times are larger than `T`, here we set it to `Inf` as we don't want it to be the stopping criterion but rather want to set a number of gradient evaluations.
* if the number of gradient evaluation is larger than `maxgradeval`

Other parameters:

* $\lambda_{\text{ref}}$ is the refreshment rate (see documentation of `PDMP.jl`)
* $x_0, v_0$ are the initial starting points and velocity
* `sim` encapsulates all the parameters

Then, the only thing left to do is to run the simulation.

In [20]:
T           = Inf      
maxgradeval = 1000*1000 # these are CV gradients so very cheap
λ_ref       = 2.

x0  = rand(dm.p)
v0  = randn(dm.p)
v0 /= norm(v0)

sim = PDMP.Simulation(x0, v0, T, nextevent,
                      gll_cv, nextboundary, λ_ref;
                      maxgradeval = maxgradeval)

(path, details) = PDMP.simulate(sim)
;

### Having a look at the results

#### Simulation details

The `details` dictionary contains informations about the simulation such as the number of boundary hits (`nboundary`), the clocktime, the number of refreshments etc.

In [21]:
details

Dict{String,Any} with 7 entries:
  "nboundary" => 315
  "nsegments" => 1103
  "clocktime" => 37.1363
  "nloops"    => 1000370
  "nbounce"   => 732
  "ngradeval" => 1000000
  "nrefresh"  => 55

#### Path object

The `path` object corresponds to the path generated with the simulation. You could for example:

* sample it using the `samplepath` function
* obtain the mean for each dimension via the `pathmean` function
* obtain the ess for each dimension via the `esspath` function

(refer to `PDMP.jl` for more informations)

**Note**: when using Julia 0.6, using `esspath` may show a warning, this is because it relies on the package `Klara.jl` which uses deprecated syntax. You can safely ignore that warning. If you'd like to not see such warnings, run this:

```julia
using IJulia
IJulia.installkernel("Julia nodeps", "--depwarn=no")
```

then restart the notebook and change the kernel to Julia nodeps (see also `README.md`)

In [22]:
# Compute the RMSE
ypred = PDMP.logistic.(ξ * PDMP.pathmean(path)) * 2 - 1
rmse  = sqrt(sum((ypred-y).^2)/length(y))
println("RMSE: $(round(rmse,3))")

# Show the average ESS across dimensions
all_ess = PDMP.esspath(path)[1]
println("Min ESS: $(round(minimum(all_ess),1))")
println("Max ESS: $(round(maximum(all_ess),1))")

RMSE: 0.153
Min ESS: 9.5
Max ESS: 55.8
